# Tutorial 03 - Model Loading

This tutorial demonstrates how to create tensorflow models for supervised and weakly trainings using the paws API.

In [1]:
import os
from paws.components import ModelLoader

# here outdir is the directory to which model outputs are saved, this is only needed for loading
# semi-weakly models which is built from supervised models (it needs to know where to load the
# supervised models)
outdir = "/pscratch/sd/c/chlcheng/projects/paws/outputs"

# for more details of the parameters, use help(ModelLoader.__init__)
model_loader = ModelLoader(feature_level="high_level", decay_modes='qq,qqq',
                           variables="3,5,6", noise_dimension=0, outdir=outdir)

In [2]:
from paws.components import DataLoader

datadir = "/pscratch/sd/c/chlcheng/projects/paws/datasets"

# data loading is needed to get the shape and dtype of the train features
data_loader = DataLoader(datadir=datadir, feature_level="high_level", decay_modes='qq,qqq',
                         variables="3,5,6", noise_dimension=0, seed=2023)
datasets = data_loader.get_datasets(mass_point=[300, 300])

2024-06-01 21:33:56.967190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 21:33:56.968193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 21:33:57.070216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-01 21:33:57.307532: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 21:34:00.007541: W tensorflow/compiler/tf2

[INFO] Number of events in each dataset split:
            train     val    test
QCD         60700   30327   30325
extra_QCD  306450  153208  153200
W_qq        37250   18622   18600
W_qqq       38111   19050   19050
total      442511  221207  221175
num_batch     433     217     216


In [3]:
feature_metadata = data_loader.feature_metadata
feature_metadata

{'jet_features': {'shape': [2, 7], 'dtype': 'float64'},
 'param_masses': {'shape': [2], 'dtype': 'float64'},
 'label': {'shape': [1], 'dtype': 'int64'}}

### Dedicated supervised model (also used for ideal weakly training)

In [4]:
fs_model_dedicated = model_loader.get_supervised_model(feature_metadata, parametric=False)
fs_model_dedicated.summary()

Model: "HighLevel"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 jet_features (InputLayer)   [(None, 2, 3)]               0         []                            
                                                                                                  
 tf.compat.v1.shape (TFOpLa  (3,)                         0         ['jet_features[0][0]']        
 mbda)                                                                                            
                                                                                                  
 tf.__operators__.getitem (  (2,)                         0         ['tf.compat.v1.shape[0][0]']  
 SlicingOpLambda)                                                                                 
                                                                                          

### Parameterised supervised model

In [5]:
fs_model_parameterised = model_loader.get_supervised_model(feature_metadata, parametric=True)
fs_model_parameterised.summary()

Model: "HighLevel"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 param_masses (InputLayer)   [(None, 2)]                  0         []                            
                                                                                                  
 jet_features (InputLayer)   [(None, 2, 3)]               0         []                            
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 2, 1)                 0         ['param_masses[0][0]']        
 )                                                                                                
                                                                                                  
 tf.concat (TFOpLambda)      (None, 2, 4)                 0         ['jet_features[0][0]',

### Semi-weakly model

In [6]:
# get paths to the supervised models
fs_ckpt_dir = model_loader.get_checkpoint_dir("param_supervised", decay_mode="qq", split_index=0, version="v1")
fs_model_path = os.path.join(fs_ckpt_dir, model_loader.path_manager.get_basename('model_full_train'))
fs_ckpt_dir_2 = model_loader.get_checkpoint_dir("param_supervised", decay_mode="qqq", split_index=0, version="v1")
fs_model_path_2 = os.path.join(fs_ckpt_dir_2, model_loader.path_manager.get_basename('model_full_train'))

ws_model = model_loader.get_semi_weakly_model(feature_metadata, fs_model_path=fs_model_path,
                                              fs_model_path_2=fs_model_path_2)
ws_model.summary()

Model: "SemiWeakly"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 jet_features (InputLayer)   [(None, 2, 3)]               0         []                            
                                                                                                  
 tf.ones_like (TFOpLambda)   (None, 2, 3)                 0         ['jet_features[0][0]']        
                                                                                                  
 tf.ones_like_1 (TFOpLambda  (None, 2, 3)                 0         ['jet_features[0][0]']        
 )                                                                                                
                                                                                                  
 tf.__operators__.getitem_2  (None,)                      0         ['tf.ones_like[0][0]'

### Train setup and configurations

In [9]:
# get path to checkpoint directory based on the desired scenario
ws_ckpt_dir = model_loader.get_checkpoint_dir("semi_weakly", split_index=0, version="v1",
                                              mass_point=[300, 300], mu=0.005, alpha=0.5,
                                              trial=0)
# get train configuration
train_config = model_loader.get_train_config(checkpoint_dir=ws_ckpt_dir, model_type='semi_weakly')
train_config

{'loss': <aliad.interface.tensorflow.losses.ScaledBinaryCrossentropy at 0x7fb72878f390>,
 'metrics': ['accuracy'],
 'epochs': 100,
 'optimizer': 'Adam',
 'optimizer_config': {'learning_rate': 0.05,
  'clipvalue': 0.0001,
  'clipnorm': 0.0001},
 'checkpoint_dir': '/pscratch/sd/c/chlcheng/projects/paws/outputs/train_results/semi_weakly/high_level/qq_qqq/300_300/SR_var_3_5_6_noise_0_v1/mu_0p005_alpha_0p5/split_0/trial_0',
 'callbacks': {'lr_scheduler': {'initial_lr': 0.05,
   'lr_decay_factor': 0.5,
   'patience': 5,
   'min_lr': 1e-06,
   'verbose': True},
  'early_stopping': {'monitor': 'val_loss',
   'patience': 20,
   'restore_best_weights': True},
  'model_checkpoint': {'save_weights_only': True, 'save_freq': 'epoch'},
  'metrics_logger': {'save_freq': -1},
  'weights_logger': {'save_freq': -1, 'display_weight': True}}}

In [11]:
# get list of callbacks used in training
callbacks = model_loader.get_callbacks(model_type='semi_weakly', config=train_config)
callbacks

{'early_stopping': <aliad.interface.tensorflow.callbacks.EarlyStopping at 0x7fb7287c9590>,
 'model_checkpoint': <keras.src.callbacks.ModelCheckpoint at 0x7fb7287caad0>,
 'metrics_logger': <aliad.interface.tensorflow.callbacks.MetricsLogger at 0x7fb7287cae90>,
 'lr_scheduler': <aliad.interface.tensorflow.callbacks.LearningRateScheduler at 0x7fb7287c9f90>,
 'weights_logger': <aliad.interface.tensorflow.callbacks.WeightsLogger at 0x7fb7287ca310>}

In [12]:
# compile model
model_loader.compile_model(ws_model, train_config)

In [13]:
# manually set trainable parameters in the semi-weakly model
# note the unit of m1 and m2 here is 10GeV, mu is the natural log of actual signal fraction
import numpy as np
model_loader.set_semi_weakly_model_weights(ws_model, m1=0.5, m2=0.5, mu=np.log(1e-3), alpha=0.5)

In [15]:
model_loader.get_semi_weakly_model_weights(ws_model)

{'m1': 0.5, 'm2': 0.5, 'alpha': 0.5, 'mu': -6.9077554}